In [1]:
# importing necessary libraries

from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import os

import cv2 
import tensorflow as tf
import keras

from keras.utils import plot_model,np_utils

from keras.layers.convolutional import Conv3D
from keras.layers.pooling import MaxPooling3D, AveragePooling3D
from keras.initializers import Constant

from keras.layers import Conv3D, MaxPool3D, Flatten, Dense, Layer
from keras.layers import Dropout, Input, BatchNormalization, concatenate

from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta, SGD
from keras.models import Model

from keras.callbacks import LearningRateScheduler
from keras import backend as K 

from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

from keras.layers import *
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

import h5py
import math 

# from plotly.offline import iplot, init_notebook_mode
# import plotly.graph_objs as go
# init_notebook_mode(connected=True)
%matplotlib inline


In [2]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 3
MAX_SEQ_LENGTH = 8

In [3]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    frames=frames[:MAX_SEQ_LENGTH]
    return np.array(frames)

In [4]:
train_dir = "../vid_data/train/"
test_dir = "../vid_data/test/"
classes_list = os.listdir(train_dir)

In [5]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(classes_list)
)
print(label_processor.get_vocabulary())


# labels = classes_list
# print(labels)
# labels = np.array(labels) 
# print(labels)
# convert classlabels to label encoding
labels = label_processor(classes_list).numpy()
print(labels)

labels.shape

['catch', 'climb']
[0 1]


c:\Users\Admin\.conda\envs\iNile_dipr\lib\site-packages\numpy\core\numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


(2,)

In [6]:
def prepare_all_videos(root_dir):
    frame = []
    labels = []
    class_labels = []
    
    classes_list = os.listdir(root_dir)
    video_file_path = []
    for class_name in classes_list:
        class_dir = os.path.join(root_dir, class_name)
        
        for file_name in os.listdir(class_dir):
            video_file_path.append(os.path.join(root_dir, class_name, file_name))
            labels = []
            labels.extend([class_name])
            class_labels.append(labels)
        # class_labels.append([[class_name]*MAX_SEQ_LENGTH])
    # class_labels = [[i]*MAX_SEQ_LENGTH for i in classes_list]
    # labels = np.array(classes_list)
    class_labels = np.array(class_labels)
    # print(class_labels.shape)
    # print(class_labels)
    class_labels = label_processor(class_labels).numpy()
    # print(class_labels)
    # For each video.
    for idx, path in enumerate(video_file_path):
        # print(path)
        frames = load_video(path)
        frame.append(frames)
       

        
    return frame, class_labels


train_data, train_labels = prepare_all_videos(train_dir)
test_data, test_labels = prepare_all_videos(test_dir)

train_data = np.array(train_data)
test_data = np.array(test_data)

print(f"train_data in train set: {len(train_data)}")
print(f"train_data in train set: {train_data.shape}")
print(f"train_labels in train set: {train_labels.shape}")
print(train_data)
print(train_labels)

print(f"test_data in train set: {len(test_data)}")
print(f"test_data in train set: {test_data.shape}")
print(f"test_labels in train set: {test_labels.shape}")

#train_data each index: (20, 224, 224, 3)
print(f"train_data each index: {train_data[0].shape}")
print(f"test_data each index: {test_data[0].shape}")



train_data in train set: 210
train_data in train set: (210, 8, 224, 224, 3)
train_labels in train set: (210, 1)
[[[[[196 215 246]
    [193 212 242]
    [191 214 239]
    ...
    [252 252 252]
    [252 252 252]
    [252 252 252]]

   [[195 215 246]
    [192 213 242]
    [191 215 240]
    ...
    [252 252 252]
    [252 252 252]
    [252 252 252]]

   [[193 218 244]
    [190 216 241]
    [191 219 242]
    ...
    [252 252 252]
    [252 252 252]
    [252 252 252]]

   ...

   [[ 29  43  11]
    [ 24  38   6]
    [ 22  36   4]
    ...
    [ 58  61  72]
    [ 68  73  81]
    [ 54  59  67]]

   [[ 30  44  12]
    [ 25  39   7]
    [ 23  37   5]
    ...
    [ 40  43  54]
    [ 28  31  40]
    [ 30  33  42]]

   [[ 27  41   9]
    [ 31  45  13]
    [ 30  44  12]
    ...
    [ 32  35  46]
    [ 28  31  40]
    [ 28  31  40]]]


  [[[196 215 246]
    [193 212 242]
    [191 214 239]
    ...
    [252 252 252]
    [252 252 252]
    [252 252 252]]

   [[195 215 246]
    [192 213 242]
    [191 215 240

In [8]:
def load_video_for_flow(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            # frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    frames=frames[:MAX_SEQ_LENGTH]
    return frames


def prepare_optical_flow(root_dir):
    frame = []
    f_labels = []
    flow_labels = []
    optical_flow_motion = []
    
    classes_list = os.listdir(root_dir)
    video_file_path = []
    for class_name in classes_list:
        class_dir = os.path.join(root_dir, class_name)
        
        for file_name in os.listdir(class_dir):
            video_file_path.append(os.path.join(root_dir, class_name, file_name))
            f_labels = []
            f_labels.extend([class_name])
            flow_labels.append(f_labels)
        
    flow_labels = np.array(flow_labels)
    
    flow_labels = label_processor(flow_labels).numpy()
   
    # For each video.
    
    for idx, path in enumerate(video_file_path):
        # print(path)
        frames = load_video_for_flow(path)
        flow = []
        prev = frames[0]
        prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
        for i, frame_curr in enumerate(frames):
            
            curr = frame_curr
            curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
            flow_tmp = cv2.calcOpticalFlowFarneback(prev, curr, None, 0.6, 3, 25, 7, 5, 1.2, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
            # print(flow_tmp.shape)
            flow.append(flow_tmp)
            prev = curr
        
        optical_flow_motion.append(flow)
       

        
    return optical_flow_motion, flow_labels


of_train_data, of_train_labels = prepare_optical_flow(train_dir)
of_test_data, of_test_labels = prepare_optical_flow(test_dir)

of_train_data = np.array(of_train_data)
of_test_data = np.array(of_test_data)

print(f"optical_flow_train_data in train set: {len(of_train_data)}")
print(f"optical_flow_train_data in train set: {of_train_data.shape}")
print(f"optical_flow_train_labels in train set: {of_train_labels.shape}")
# print(of_train_data)
# print(of_train_labels)

print(f"optical_flow_test_data in train set: {len(of_test_data)}")
print(f"optical_flow_test_data in train set: {of_test_data.shape}")
print(f"optical_flow_test_labels in train set: {of_test_labels.shape}")

#train_data each index: (20, 224, 224, 3)
print(f"optical_flow_train_data each index: {of_train_data[0].shape}")
print(f"optical_flow_test_data each index: {of_test_data[0].shape}")


(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 224, 2)
(224, 

In [ ]:
# inception module definition

def inc_module(x, f1, f3r, f3, f33r, f33, f4, name):
    '''
    this is an inception model
    '''
#     print(x.shape)
    c1 = Conv3D(f1, (1,1,1), 
                      padding="SAME",
                      activation='relu',
                      kernel_initializer=kernel_init,
                      bias_initializer=bias_init)(x)
#     print(c1.shape)
    c2 = Conv3D(f3r, (1,1,1),
                      padding="SAME",
                      activation='relu',
                      kernel_initializer=kernel_init,
                      bias_initializer=bias_init)(x)
#     print(c2.shape)
    c2 = Conv3D(f3, (3,3,3), 
                      padding="SAME",
                      activation='relu',
                      kernel_initializer=kernel_init,
                      bias_initializer=bias_init)(c2)
    
#     print(c2.shape)
    c3 = Conv3D(f33r, (1,1,1), 
                      padding="SAME",
                      activation='relu',
                      kernel_initializer=kernel_init,
                      bias_initializer=bias_init)(x)
    c3 = Conv3D(f33, (3,3,3), 
                      padding="SAME",
                      activation='relu',
                      kernel_initializer=kernel_init,
                      bias_initializer=bias_init)(c3)
    
    m3 = MaxPool3D(pool_size=(3, 3, 3),
                   strides=(1, 1, 1),
                   padding="SAME")(x)
    c4 = Conv3D(f4, (1,1,1), 
                       padding="SAME",
                       activation='relu',
                       kernel_initializer=kernel_init,
                       bias_initializer=bias_init)(m3)
    
    output = concatenate([c1, c2, c3, c4], axis=4, name = name)
 
    return output

kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [ ]:
# _IMAGE_SIZE = 224
# _VIDEO_FRAMES = 8

# IMG_SIZE = 224
# BATCH_SIZE = 64
# EPOCHS = 3
# MAX_SEQ_LENGTH = 8

# input layer
vid_input = Input(shape=(MAX_SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3))

# first conv1 layer
conv1 = Conv3D(64,(7,7,7),
               strides=(2, 2, 2),
               padding="SAME",
               activation='relu',
               data_format='channels_last')(vid_input)
print(conv1.shape)
pool1 = MaxPool3D(pool_size=(1, 3, 3),
                     strides=(1, 2, 2),
                     padding="SAME")(conv1)

conv2 = Conv3D(64,(1,1,1),
               padding="SAME",
               activation='relu')(pool1)

conv3 = Conv3D(192,(3,3,3),
               padding="SAME",
               activation='relu')(conv2)

pool2 = MaxPool3D(pool_size=(1, 3, 3),
                     strides=(1, 2, 2),
                     padding="SAME")(conv3)

print(pool2.shape)
# INC_a
inc = inc_module(pool2,f1=64,f3r=96,f3=128,f33r=16,f33=32,f4=32,name='inception_a')

# INC_b
inc = inc_module(inc,f1=128,f3r=128,f3=192,f33r=32,f33=96,f4=64,name='inception_b')

# MAXPOOL
pool3 = MaxPool3D(pool_size=(3, 3, 3), strides=(2, 2, 2), padding="SAME")(inc)

# INC_c
inc = inc_module(pool3,f1=192,f3r=96,f3=208,f33r=16,f33=48,f4=64,name='inception_c')

# INC_d
inc = inc_module(inc,f1=160,f3r=112,f3=224,f33r=24,f33=64,f4=64,name='inception_d')

# INC_e
inc = inc_module(inc,f1=128,f3r=128,f3=256,f33r=24,f33=64,f4=64,name='inception_e')

# INC_f
inc = inc_module(inc,f1=112,f3r=144,f3=288,f33r=32,f33=64,f4=64,name='inception_f')

# INC_g
inc = inc_module(inc,f1=256,f3r=160,f3=320,f33r=32,f33=128,f4=128,name='inception_g')

# MAXPOOL
pool4 = MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="SAME")(inc)

# INC_h
inc = inc_module(pool4,f1=256,f3r=160,f3=320,f33r=32,f33=128,f4=128,name='inception_h')

# INC_i
inc = inc_module(inc,f1=384,f3r=192,f3=384,f33r=48,f33=128,f4=128,name='inception_i')

# AVERAGE POOL
pool5 = AveragePooling3D(pool_size=(2, 7, 7), strides=(1, 1, 1), padding="SAME")(inc)

# DROPOUT 1.0 spatial_squeeze=True

conv4 = Conv3D(51,(1,1,1),
               padding="SAME",
               activation='relu')(pool5)

b_norm = BatchNormalization()(conv4)
flatten_layer = Flatten()(b_norm)

dense_layer1 = Dense(units=2048, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=512, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)

# flatten_layer_last = Flatten()()
# output = Dense(2, activation='softmax')(dense_layer2)
output = Dense(1, activation='sigmoid')(dense_layer2)
model = Model(inputs=vid_input, outputs=output)


(None, 4, 112, 112, 64)
(None, 4, 28, 28, 192)


In [ ]:
 #Adding Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss',
                                        patience = 15, 
                                        mode = 'min', 
                                        restore_best_weights = True)

# Adding loss, optimizer and metrics values to the model.
model.compile(loss='binary_crossentropy',
            # loss = 'categorical_crossentropy', 
            # loss = 'sparse_categorical_crossentropy',
              optimizer = 'Adam', 
              metrics = ["accuracy"])

# Start Training
model_training_history = model.fit(x = train_data,
                                   y = train_labels, 
                                   epochs = 3, 
                                   batch_size = 4, 
                                   shuffle = True, 
                                   validation_split = 0.2, 
                                   callbacks = [early_stopping_callback])

Epoch 1/3


2022-10-12 12:44:29.025536: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 206524416 exceeds 10% of free system memory.


1/8 [==>...........................] - ETA: 2:22 - loss: 0.6366 - accuracy: 0.5000

2022-10-12 12:44:45.955448: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 206524416 exceeds 10% of free system memory.


2/8 [======>.......................] - ETA: 1:41 - loss: 1.5355 - accuracy: 0.6250

...

3/8 [==========>...................] - ETA: 1:24 - loss: 1.3634 - accuracy: 0.6667

2022-10-12 12:45:20.027121: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 206524416 exceeds 10% of free system memory.


4/8 [==============>...............] - ETA: 1:08 - loss: 1.2578 - accuracy: 0.5625

2022-10-12 12:45:37.054909: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 206524416 exceeds 10% of free system memory.


8/8 [==============================] - 141s 17s/step - loss: 1.2873 - accuracy: 0.5938 - val_loss: 593.4692 - val_accuracy: 0.0000e+00
Epoch 2/3
8/8 [==============================] - 137s 17s/step - loss: 2.3609 - accuracy: 0.6250 - val_loss: 514.6355 - val_accuracy: 0.0000e+00
Epoch 3/3
8/8 [==============================] - 136s 17s/step - loss: 2.8502 - accuracy: 0.6875 - val_loss: 410.1638 - val_accuracy: 0.0000e+00


In [ ]:
score = model.evaluate(test_data, test_labels, verbose = 0)
score

[187.50521850585938, 0.5]

In [ ]:
# load_video(path)
path_for_pred = "/home/nilesh/Desktop/har_model/vid_data/test/Dan_Osman_-_Speed_rock_climbing_climb_f_cm_np1_fr_bad_5.avi"
pred_frame = load_video(path_for_pred)
# model.predict(pred_frame)

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.